In this lab, I first normalized the data given, ran logistic regression with gradient descent, then calcuated the errror rates for the training and testing data.

In [2]:
%pylab inline
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


Reading in training and testing data

In [2]:
training_data = pd.read_csv('spam-train.txt',header=None)
testing_data = pd.read_csv('spam-test.txt',header=None)

In [3]:
training_data.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.0,0.00,0.00,0.0,1.01,0.0,0.0,0.0,0.00,0.00,...,0.000,0.088,0.000,0.000,0.088,0.000,6.718,33.0,215.0,1.0
1,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,...,0.098,0.589,0.000,0.000,0.000,0.000,2.044,22.0,92.0,1.0
2,0.0,0.53,0.00,0.0,1.06,0.0,1.6,0.0,0.00,0.53,...,0.000,0.239,0.079,0.159,0.000,0.000,4.555,51.0,123.0,1.0
3,0.0,0.00,0.23,0.0,0.92,0.0,0.0,0.0,0.23,0.00,...,0.000,0.130,0.026,0.026,0.000,0.026,2.222,23.0,480.0,0.0
4,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,1.428,4.0,10.0,0.0


In [4]:
testing_data.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,14.28,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,1.800,5.0,9.0,0.0
1,0.51,0.43,0.29,0.0,0.14,0.03,0.00,0.18,0.54,0.62,...,0.012,0.078,0.000,0.443,0.510,0.133,6.590,739.0,2333.0,1.0
2,0.79,0.19,0.09,0.0,0.00,0.00,0.00,0.09,0.29,0.09,...,0.000,0.267,0.000,0.190,0.247,0.000,2.324,19.0,365.0,1.0
3,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.151,0.227,0.075,0.075,0.075,0.000,2.482,12.0,139.0,0.0
4,0.17,0.17,0.25,0.0,0.43,0.08,0.08,0.08,0.69,2.41,...,0.000,0.107,0.000,0.308,0.067,0.026,4.215,82.0,1214.0,1.0


Splitting the data into the data with features and the actual classification

In [5]:
Xtrain = training_data.drop(57,axis=1)
Ytrain = training_data[:][57]

Xtest = testing_data.drop(57,axis=1)
Ytest = testing_data[:][57]

Function to normalize the data

In [6]:
def z_score(data):
    zscore_Xtrain = data.copy()
    zscore_Xtrain = (zscore_Xtrain-zscore_Xtrain.mean(axis = 0))/zscore_Xtrain.std(axis = 0)
    return zscore_Xtrain

In [7]:
z_score(Xtrain).head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,-0.341197,-0.167892,-0.559543,-0.047041,0.990915,-0.358135,-0.279760,-0.274942,-0.318200,-0.395557,...,-0.114562,-0.160496,-0.219886,-0.164386,-0.449056,0.028504,-0.099663,0.054088,-0.129557,-0.114247
1,-0.341197,-0.167892,-0.559543,-0.047041,-0.450720,-0.358135,-0.279760,-0.274942,-0.318200,-0.395557,...,-0.114562,0.244309,2.059477,-0.164386,-0.449056,-0.297565,-0.099663,-0.091047,-0.215281,-0.306383
2,-0.341197,0.259688,-0.559543,-0.047041,1.062283,-0.358135,3.589996,-0.274942,-0.318200,0.460440,...,-0.114562,-0.160496,0.467108,0.603462,-0.169589,-0.297565,-0.099663,-0.013076,0.010717,-0.257958
3,-0.341197,-0.167892,-0.113615,-0.047041,0.862453,-0.358135,-0.279760,-0.274942,0.505942,-0.395557,...,-0.114562,-0.160496,-0.028801,0.088323,-0.403357,-0.297565,-0.037943,-0.085520,-0.207488,0.299705
4,-0.341197,-0.167892,-0.559543,-0.047041,-0.450720,-0.358135,-0.279760,-0.274942,-0.318200,-0.395557,...,-0.114562,-0.160496,-0.620253,-0.164386,-0.449056,-0.297565,-0.099663,-0.110175,-0.355555,-0.434474


Sigmoid function used in logistic regression and for prediction in the testing phase

In [8]:
def sigmoid(x):
    sigmoid_value = 1./(1 + math.exp(-x))
    return sigmoid_value

vectorized_sigmoid = np.vectorize(sigmoid)

Logistic Regression Model

In [9]:
class logistic_regression:  
    def fit(self,Xtrain,Ytrain):
        X = Xtrain.copy()
        X['57'] = np.ones(X.shape[0])
        
        w0 = np.random.normal(0,1,size=X.shape[1])
        
        difference = 1
        while difference > 0.00001:
            #right = np.dot((vectorized_sigmoid(np.dot(X,w0)) - Ytrain), X)
            #wPlus = w0 - 0.00001*right
            wPlus = w0 + 0.00001 * np.dot(X.T,(Ytrain - vectorized_sigmoid(np.dot(X,w0))))
            difference = np.linalg.norm(wPlus - w0) 
            w0 = wPlus  
        self.w0 = w0
        return self
    
    def prediction(self,Xtest):
        X = Xtest.copy()
        X['57'] = np.ones(X.shape[0]) 
        predictions = vectorized_sigmoid(np.dot(self.w0,X.T))
        return predictions

In [10]:
lr = logistic_regression()
lr.fit(z_score(Xtrain),Ytrain)

In [11]:
train_predictions = lr.prediction(z_score(Xtrain))
test_predictions = lr.prediction(z_score(Xtest))

Function to classify as spam or not spam based on the value returned from the probability function

In [12]:
prediction_train = []
prediction_test = []
for i in train_predictions:
    if i > 0.5:
        prediction_train.append(1)
    else:
        prediction_train.append(0)

for j in test_predictions:
    if j > 0.5:
        prediction_test.append(1)
    else:
        prediction_test.append(0) 

In [13]:
def error_rate(Y,predictions):
    count = 0
    for i in range(len(Y)):
        if predictions[i] != Y[i]:
            count += 1
    error_rate = count/len(Y)
    return error_rate

Error Rate for Training Data

In [14]:
error_rate(Ytrain,prediction_train)

0.06688417618270799

Error Rate for Testing Data

In [15]:
error_rate(Ytest,prediction_test)

0.06770833333333333